# Washington State DOT Traffic API

source: [https://wsdot.wa.gov/traffic/api](https://wsdot.wa.gov/traffic/api)

In [30]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv
from time import sleep
from sqlalchemy import create_engine, text

## Import env variables

In [6]:
# load the .env file
load_dotenv()

# API variables
api_key = os.getenv("API_KEY")

# MySQL database variables
db_user = os.getenv("AZURE_USERNAME")
db_pwd = os.getenv("AZURE_PWD")
db_host = os.getenv("AZURE_URL")
db_port = os.getenv("AZURE_PORT")
db_database = os.getenv("AZURE_DB")

## URLs to Acces APIs

In [7]:
TRAVEL_TIMES_URL = "http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}"

TRAFFIC_ALERTS_URL = "http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}"

WEATHER_INFORMATION_URL = "http://wsdot.wa.gov/Traffic/api/WeatherInformation/WeatherInformationREST.svc/GetCurrentWeatherInformationAsJson?AccessCode={ACCESSCODE}"

In [16]:
# customer function to get api data
def get_api_data(url, access_key):
    # create the url with the access key
    url_api = url.format(ACCESSCODE=access_key)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        # print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    # return the dataframe and the status code
    return df, response.status_code


# function to convert dictionary columns from api to json
def convert_dict_to_json(dataframe):

    for col in dataframe.columns:
        if dataframe[col].apply(lambda x: isinstance(x, dict)).all():
            dataframe[col] = dataframe[col].apply(lambda x: json.dumps(x))

In [32]:
def upload_to_history_table(dataframe, table_name, conn):

    try:
        dataframe.to_sql(
            table_name,
            con=conn,
            if_exists="append",
            index=False,
        )
        print(f"{table_name} uploaded successfully.")
    except Exception as e:
        print(f"Failed to upload {table_name}. Error: {e}")


def update_raw_table(dataframe, table_name, conn):

    try:
        dataframe.to_sql(
            table_name,
            con=conn,
            if_exists="replace",
            index=False,
        )
        print(f"{table_name} uploaded successfully.")
    except Exception as e:
        print(f"Failed to upload {table_name}. Error: {e}")

In [20]:
# function to get api data
def api_update(
    travel_times_url, traffic_alerts_url, weather_information_url, access_key
):

    # Get current time
    timestamp = pd.Timestamp.now()
    df_tt, response_tt = get_api_data(travel_times_url, access_key)
    df_ta, response_ta = get_api_data(traffic_alerts_url, access_key)
    df_wa, response_wa = get_api_data(weather_information_url, access_key)

    if response_tt == 200 and response_ta == 200 and response_wa == 200:
        status = "Success"
    else:
        status = "Failed"

    # add pull_date_time
    df_tt["timestamp"] = timestamp
    df_ta["timestamp"] = timestamp
    df_wa["timestamp"] = timestamp

    # convert dict columns to json
    convert_dict_to_json(df_tt)
    convert_dict_to_json(df_ta)
    convert_dict_to_json(df_wa)

    df_api_fetch = pd.DataFrame(
        [
            {
                "timestamp": timestamp,
                "travel_times_response": response_tt,
                "traffic_alerts_response": response_ta,
                "weather_alerts_response": response_wa,
                "status": status,
            }
        ]
    )

    return df_api_fetch, df_tt, df_ta, df_wa, status

# Extract and Load

In [37]:
def extract_load_transform():
    SLEEP_TIME = 5
    # get the data
    df_api_fetch, df_tt, df_ta, df_wa, status = api_update(
        TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
    )

    # connect to dateabase
    connection_url = (
        f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
    )

    try:
        engine = create_engine(connection_url)
        with engine.connect() as conn:
            print("*** Connected to database ***")

            # Append history tables
            print("*** Appending to history tables ***")
            upload_to_history_table(df_api_fetch, "api_fetch_hist", conn)
            upload_to_history_table(df_tt, "time_travel_hist", conn)
            upload_to_history_table(df_ta, "traffic_alerts_hist", conn)
            upload_to_history_table(df_wa, "weather_alerts_hist", conn)

            if status == "Success":

                print("*** Updating raw tables ***")
                update_raw_table(df_api_fetch, "api_fetch", conn)
                update_raw_table(df_tt, "time_travel_raw", conn)
                update_raw_table(df_ta, "traffic_alerts_raw", conn)
                update_raw_table(df_wa, "weather_alerts_raw", conn)

                # Wait a few seconds then call stored procedures
                sleep(SLEEP_TIME)
                tt_transform_result = conn.execute(text("CALL TransformTravelTime"))
                ta_transform_result = conn.execute(text("CALL TransformTrafficAlerts"))
                wa_transform_result = conn.execute(text("CALL TransformWeatherAlerts"))
    except Exception as e:
        print("Connection unsuccessful. Error: ", e)

    print(tt_transform_result)

    return None


extract_load_transform()

*** Connected to database ***
*** Appending to history tables ***
api_fetch_hist uploaded successfully.
time_travel_hist uploaded successfully.
traffic_alerts_hist uploaded successfully.
weather_alerts_hist uploaded successfully.
*** Updating raw tables ***
api_fetch uploaded successfully.
time_travel_raw uploaded successfully.
traffic_alerts_raw uploaded successfully.
weather_alerts_raw uploaded successfully.
Connection unsuccessful. Error:  (pymysql.err.OperationalError) (1054, "Unknown column 'raw.id' in 'field list'")
[SQL: CALL TransformWeatherAlerts]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
